In [1]:
# needed to load the REBEL model
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
import math
import torch

# wrapper for wikipedia API
# scraping of web articles
# google news scraping

# graph visualization

# show HTML in notebook
import IPython

from keybert import KeyBERT

from sentence_transformers import SentenceTransformer
import numpy as np
import nltk
import re
from typing import List, Tuple, Union

import spacy

import inspect
import os

import ollama
from tqdm import tqdm
import time
import re
import sys
import pandas as pd
from openie import StanfordOpenIE

module_path = 'core'
sys.path.append(module_path)

from SemanticSimilarity import SemanticSimilarity
from KeywordExtractor import KeywordExtractor
from TripletGenerator import TripletGenerator
from KG import KG

In [13]:
pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.max_rows', None)     # Display all rows


In [15]:
def retrieve_text_from_filepath(filepath):
    with open(filepath,'r') as f:
        t = f.read()
        return t


In [16]:
text = """﻿
                                1 of 8 DOCUMENTS

                               The New York Times

                 June 1, 1980, Sunday, Late City Final Edition

LANDLORD-TENANT TENSION RISES OVER ISSUES OF RIGHTS AND RULES

BYLINE: By WILLIAM G. BLAIR

SECTION: Section 8; Page 1, Column 3; Real Estate Desk

LENGTH: 1926 words


Complaints against landlords in New York City are sharply on the rise, according
to public and private agencies concerned with the problem. The majority of the
complaints are about alleged overcharging, but apparently stricter landlord
enforcement of rental leases is also prompting inquiries from tenants.

Frank S. Kristof, president of a group that represents owners of rent-stabilized
apartments, said the association was getting 600 calls a day, compared with 20
or 30 a day not too long ago. He said that 62 percent of the calls were from
tenants and 38 percent from owners. He explained that he thought the upsurge was
''directly related to the tightness of the market.'' Landlords, he said, are
being ''sorely tempted by the eagerness of tenants to find homes.''

Jane Benedict, chairman and a founder of the 21-year-old Metropolitan Council of
Housing, the biggest citywide tenant union, said the council ''became aware
about a year ago of an accelerated effort by landlords to enforce technical
aspects of leases.''

''Our phones started jumping off their hooks,'' she said, ''and now we're
getting a flood of complaints,'' from increasingly organized tenants. By
invoking ''all kinds of technical matters,'' she said, the owners hope to ''get
rid of'' the existing occupant and bring in a new tenant at an increased rent.

Ellis Franke, executive director of the City Conciliation and Appeals Board,
which adjudicates tenant-landlord disputes in rentstabilized apartments, said
that complaints of overcharging in the six-month period ending in March were 47
percent greater than in the preceding half-year.

She added that complaints were ''now coming in at a rate that will be in the
neighborhood of 4,000 a year, and that's double what it was two years ago.''

Stephen Mindell, an assistant attorney general in the state Attorney General's
office, said it was ''very disturbing to us to find that in a substantial number
of cases, the self-regulating system of rent stabilization does not appear to be
working.'' Mr.  Mindell is in charge of a continuing investigation into rent
overcharges begun by the attorney general's office in February 1979.  His
office, he said, has obtained $700,000 in refunds to tenants.

New York City's rental housing essentially operates under two separate systems:
the Rent Stabilization Law for 850,000 apartments and the Rent Control Law for
400,000 others. Rent stabilization, which took effect in 1969, was fashioned to
be largely selfregulating. Rent control, which began under Federal auspices in
World War II and then was taken over by states and localities, is administered
here by the city government.

Rent increases in rent-stabilized apartments are set each year by the Rent
Guidelines Board. Under rent control, the city must approve any building's rent
increase. There is no public registration of leases for rent-stabilized
apartments to which a tenant can turn.  But for rent-controlled apartments, the
city keeps public records of rents.

Virtually everyone involved in rental housing problems here agrees that the
combination of two laws with their complexity, particularly that of rent
stabilization, leads to a large measure of ignorance about their rights and
obligations by both tenants and landlords.  Efforts are under way to reform and
consolidate the systems, but the problem is a highly complicated and emotional
one that will not lend itself easily to solution.

As of last month, all new and renewal leases for such apartments must henceforth
include a rider that advises tenants and owners of their rights and obligations
under the Rent Stabilization Law. But the rider, which covers such potentially
contentious points as permissible rent increases, subletting, services, eviction
and cooperative and condominium conversion, stresses that it is only
informational and is not a part of the lease.

Reform of the lease itself is the aim of a joint effort now being undertaken by
Mr. Mindell, the assistant attorney general, and the New York Real Estate Board,
which provides the standard lease used by many landlords in Manhattan. However,
landlords are free to modify the lease. Mr. Mindell said he was seeking plain
language in the standard lease as required by the state's Plain English Law.

Secondly, he said, his office is also trying to ''reform some of the lease
clauses to equalize the bargaining position of the tenant vis-a-vis the landlord
in light of the current law.'' He expressed optimism about the outcome of the
negotiations with the board.

Miss Franke said she hoped that including the rights statement in leases would
help matters because the bulk of the cases coming before the Conciliation and
Appeals Board ''involve new tenants who don't know the rental history of the
apartment they are taking.'' Without this history, now incorporated in the
rights statement, the tenant ''is absolutely in the dark'' about what the rent
should be, she said.

Increasingly, she said of new-tenant rent dispute cases, the board has been
getting fewer rental histories from owners ''who are unwilling or unable to
provide them - and more are unwilling than unable.''

In this sort of impasse, Miss Franke explained, the landlords frequently
default. Then, she said, ''we issue an interim order which rolls back the rent
by the applicable current rent guidelines plus 15 percent.'' The 15 percent is
the new-vacancy allowance that landlords are permitted to charge new tenants who
have signed or are about to sign new leases in the five-month period of Feb. 1
to June 30.

Before embarking on what can be a drawn-out and frustrating process of
adjudication before the Conciliation and Appeals Board or the city's Housing
Court, a tenant or landlord can try to iron out differences by seeking the
informal mediation services of the Rent Stabilization Association. The
association recently moved into new quarters with a much greater capacity to
handle the quickening flow of telephone inquiries and complaints.

Mr. Kristof, its president, said the association was settling 15 to 20 refund
disputes a week. ''When we ascertain there has been an overcharge,'' he said,
''in the overwhelming number of cases they are settled by owners with refunds to
tenants.'' When it cannot mediate successfully, he added, the association refers
the dispute to the Conciliation and Appeals Board.

Last November Sheldon Katz, chairman of the association, announced that it was
going to crack down on landlords who violated rent limits by inflicting harsh
penalties as well as requiring refunds. ''We've got to head this thing off
before it gets out of hand,'' he said.

Mr. Kristof, asked last month how many landlords had been penalized with fines
of up to $500 or suspended or expelled from the association, said, ''To date,
none.'' Under the Rent Stabilization Law, an owner suspended or expelled from
the association has his properties placed under rent control, which generally
means he must charge lower rents.

Mr. Kristof observed, however, that half a dozen cases were scheduled for
resolution by the association, none involving a major landlord. ''It's much
easier to lean on major landlords because they don't want their reputations
sullied,'' he said, adding that the association had found abuses more frequent
among smaller owners.

In addition to the overcharge and legal permissible rent problems, Mr. Kristof
said that other issues that were raised frequently included fuel cost
pass-alongs, rents for senior citizens, apartment painting and subletting or
apartment sharing.

The latter, which in current parlance is known as the roommate problem, is
described by Mrs. Benedict of the Metropolitan Council as ''a matter of economic
necessity'' for many people who cannot afford the cost of living alone.

William Moses, board chairman of CHIP (Community Housing Improvement Program)
and the owner-manager of residential buildings in Manhattan and Brooklyn, sees
the problem from a different perspective.

''While tenants have a responsibility under the lease agreement to pay rent and
take care of the apartment, people who move in illegally have none,'' Mr. Moses
said. ''When illegal roommates show up, the landlord should take action.'' He
maintained that a new lease, including a rent increase, could usually be worked
out if the tenant whose name was on the lease informed the landlord of a change
in the occupancy status.

Following the permutations of rent regulations has never been easy, but in
recent years, because of the escalating cost of fuel, the introduction of fuel
pass-alongs has further complicated an already complex situation. Miss Franke
reported that there was widespread confusion among owners and tenants about the
effective dates of passalongs, how they were computed, how long they were to
remain in effect and who was affected.

In April 1979, after stabilized rents had already been changed twice in a year
by the Rent Guidelines Board, a variable fuel surcharge of up to 2.5 percent was
added on, but only for tenants whose leases took effect between July 1, 1978,
and June 30, 1979.  The adjustment was to apply for the duration of the leases.

Before the surcharge was imposed, the Conciliation and Appeals Board was getting
about 1,000 calls a day, but after it came ''we were getting about 10,000 calls
a day,'' Miss Franke recalled. When fuel pass-alongs are applied incorrectly,
Mr. Kristof said, ''more often than not we find it's from ignorance on the part
of our landlords.''

For tenants with leases dating from July 1, 1978, to June 30, 1979, there was
still another fuel surcharge. This took effect last December when the Rent
Guidelines Board added a temporary $12-a-month surcharge to be paid for the
six-month period ending June 30. Similar action followed in March for many of
the city's rent-controlled apartments when Mayor Koch signed a fuel surcharge
measure retroactive to Jan. 1 that increased monthly rents by as much as $3.30 a
room.

Whether the $12 fuel surcharges and the increased 15 percent vacancy allowance
are to be extended beyond June 30 and in what manner are to be decided this
month by the Rent Guidelines Board. As for the annual rent control fuel
pass-along, landlords will be able to file for further increases next Jan. 1
based on any increase in fuel oil prices this year.

Some Agencies That Can Provide Assistance

Department of Housing Preservation and Development: Provides owners and tenants
with assistance on housing matters; owners, 566-3918; tenants, 566-6222.

Central Complaint Bureau: Takes telephone complaints about physical maintenance,
health, safety and sanitation. 960-4800.  Rent Guidelines Board: Information on
guideline orders. 349-2262.  Rent Stabilization Association: Provides its
owner-members with information on rent stabilization and offers mediation
services.  944-4700.

Conciliation and Appeals Board: Adjudicates disputes. 265-5920.  Metropolitan
Council on Housing: Assistance to tenants. 725-4800.

LANGUAGE: ENGLISH

GRAPHIC: Illustrations: Cartoon

                   Copyright 1980 The New York Times Company


                                2 of 8 DOCUMENTS

                               The New York Times

                 June 1, 1980, Sunday, Late City Final Edition

IT WAS A WRONG TIME AND WRONG PLACE TO BE WHITE

BYLINE: By NATHANIEL SHEPPARD Jr., Special to the New York Times

SECTION: Section 1; Part 1; Page 26, Column 1; National Desk

LENGTH: 2118 words

DATELINE: MIAMI, May 31


A blazing sun was beginning to set on another sweltering Saturday in Miami when
22-year-old Jeffrey Kult, his 18-year-old brother, Michael, and a friend,
23-year-old Deborah Getman, set out for home after a day at the beach and a
visit with friends.

A short time later, Benny Higdon, a 21-year-old bakery worker and father of
three, would call it a day at the fishing hole, load his gear in his old Dodge
Dart and head for home. With him were his 15-year-old brother-in-law, Charles
Barecca, and Robert Owens, also 15, Charles's friend from his freshman class at
high school.

The routes the two parties were to take would bring them along Northwest 62d
Street and past 12th Parkway, to a poor, rundown and predominantly black section
of Miami called Liberty City, where residents had begun to gather in groups much
larger than normal for a Saturday night. And there their paths would intersect
with an angry mob, including some angry young men who would be accused in the
coming days of beating the passers-by and killing three of them.

The Police Acquittals

About three hours earlier, in Tampa, some 200 miles away, an allwhite jury had
acquitted four whites, former policemen from Miami, of charges of using
excessive force in trying to arrest a black, Arthur McDuffie, after pursuing him
for a traffic violation.  According to testimony, Mr. McDuffie died of repeated
blows to the head by nightsticks and large flashlights.

Word of the acquittals swept through Liberty City like a tidal wave. Blacks
poured out of the hot and dilapidated houses and decrepit public housing
complexes. On the littered streets, they shouted down the Tampa verdict as an
example of how the ''system'' worked against them and for others.

Not everyone had heard of the verdict. One man, who spoke anonymously for fear
of retribution, said he and a companion were riding around the 62d Street area
as they often do on Saturday nights, ''looking to see what was happening.''

'Blow Your Horn, Brother'

The man, one of the more fortunate residents of Liberty City in that he has a
service job at a building downtown, slowed his vehicle as he neared a crowd that
had set up a roadblock across 62d Street.  He rolled down his window.

'' 'Blow your horn, brother,' '' he recalled voices from the crowd saying.
''People were putting rocks in bottles and garbage cans they had pulled to the
curb. We parked around on a side street and came back to see what was going on.

''People that came by blowing their horns or with black rags tied to their
antennas and mirrors didn't have any problem,'' he said.  ''They could get
through.''

The man recognized two men in the crowd, Leonard Joseph Capers and Lawrence
Clifford Capers, brothers, 20 and 24 years old. He said Leonard Capers was
showing off for the crowd.

It was now sometime between 6 and 7 P.M.  Jeffrey Kult was wheeling his car
along 62d Street and, unknowingly, toward the angry, waiting crowd. It was the
wrong time and wrong place to be white.

The Attack Begins

As the car approached, ''the crowd started yelling, 'Crackers, crackers' and
picking up rocks and bottles from the garbage cans,'' the man said.

The Kult automobile was pelted with stones and bottles. Suddenly the windshield
disintegrated.  ''You could see the driver trying to control the car but he
couldn't,'' the man said. ''The rocks and bottles were still flying and the
windshield was in his lap.''

''The car jumped the median,'' the man recalled, ''and headed for a little girl
and some other people standing on the side of an apartment building across the
street. I was hoping that maybe the car had missed them, but then the side of
the building turned from white to red and I knowed that somebody must have
gotten killed. People started screaming.''

''Two white people staggered out of the car,'' he said. ''One was waving his
hands like he was pleading. About 10 to 15 people started throwing things at
them. Me and my buddy knew it was time to get out of Dodge and we left.''

A Dramatic Rescue

According to police accounts of what happened then, the Kult brothers were
viciously attacked. Someone cut off one of Jeffrey's ears and part of the other,
then cut out a portion of his tongue.  Michael received a skull fracture and
other injuries from the hail of debris. Deborah Getman was lucky. A black man
whisked her to safety while the crowd's attention was on the Kults.

The brothers' lives were saved by two police officers, Manny Lopez and Arnold
Gibbs, who said they went in to make the rescue under gunfire from the crowd.

An 11-year-old girl and a 75-year-old man hit by the car were taken to
hospitals; they survived, although doctors had to amputate one of the girl's
legs.

But by this time the neighborhood had become a war zone. Stores were going up in
flames as mobs sought out and attacked all that was white or that stood as a
symbol of white dominance. The police and National Guard were called in.

The Three Fishermen

Around 9:30 P.M., apparently unaware of the uproar, Mr. Higdon and his fishing
companions drove along 62d Street, toward the crowd, which had regrouped in the
same spot near 12th Parkway.

Their car, too, was bombarded with debris. It, too, swerved out of control, and
crashed near a parking lot. The three were pulled from the car and beaten.
Again, the police came in, again under gunfire, they said, to make a rescue.
This time they were too late. The two 15-year-old boys were dead when the police
got to them. Mr. Higdon died shortly after reaching the hospital.

They were the first to die in three days of rioting that also would kill 13
more, 10 of them blacks, and leave scores injured and more than $57 million
worth of property damaged.

So far, five persons, including two 16-year-old juveniles, have been arrested
and charged with murder in the deaths of Mr. Higdon and Charles Barecca and
Robert Owens. The adults arrested were Leonard and Lawrence Capers and Nathaniel
Lane, an 18-year-old who lived two blocks from the Caperses. The police said
they had been able to make the arrests only because of the help of Liberty City
residents appalled by the slayings and violence. Three others have been accused
of assault in the beating of the Kult brothers.

Neighbors of the Capers family were reluctant to say much about them. The
children had all dropped out of high school, they said. And they mentioned that
some had criminal records.

Neighbors of Nathaniel Lane expressed surprise at his arrest. His mother, they
said, ''was a good Christian woman who always taught her children to do right.''


Difficult to Do Right

But they also spoke of how it was difficult for children to do right in such a
neighborhood as Liberty City, where it is easy to fall into bad company and
easier to steal and rob than to find a job.

The majority of Liberty City's residents are entrenched in poverty, members of a
practically unskilled, poorly educated underclass for whom the American dream is
as elusive as a rainbow.  Life in the community, where the median income is but
$271 above the Federal Government's $5,500 poverty mark, is a daily drama of the
the have-nots fighting to become haves.

The Lane family lives in a small, four-room apartment in public housing on
Northwest 60th Street. The apartment is clean and carpeted. There are pictures
of Christ and the Madonna on walls in the living room, and pictures of the four
children on walls and atop the television set. Two chairs and a sofa are
clustered around the television set and two aquariums in the room.

At the apartment, Doreen Hoskins, the mother, and Hilton Lane, the father,
talked recently about their family. Valerie Williams, 17, a friend of Nathaniel
Lane who said he was with her at her apartment on the night of the killings, was
there for the interview.

'We Are All People'

''First of all, my son is not a murderer,'' Mrs. Hoskins, a stout but genteel
woman, said sternly, pivoting to one side in the chair in which she had been
sitting while watching soap operas. ''I'm a churchgoing lady,'' she declared,
''and I don't do anything to anybody.''

''I know my son didn't kill those people,'' she said, ''because I teach my
children that regardless of whether you is black, white or green we are all
people.

''I know the Lord will make a way and all I can do is pray,'' she said
confidently. ''My son called me, crying, to say they had arrested him, and I
told him to pray and trust the Lord.''

The family has received food stamps, Mrs. Hoskins said, since she became unable
to work after an automobile accident. Their primary income is the $130 a week
that Mr. Lane brings home for driving truck for a company where he has worked 15
years.

''All I can do with that little money is to make sure they have decent clothes
on their backs,'' Mr. Lane, a small, intense man, said with some bitterness.

Nathaniel Lane dropped out of high school in his junior year to go to work, his
mother said. ''I wanted him to finish but you know how children is,'' she said.

According to court records, Nathaniel had had only minor brushes with the law.
Last October he was arrested on a charge of improperly exhibiting a weapon and
in February he was charged with loitering.  Both cases were dismissed.

Mrs. Hoskins said the family had lived in the neighborhood 20 years. ''It's no
better and no worse than any other neighborhood,'' she said. ''It's what you
make it.''

The Capers Family

The Caperses live in a dingy four-room apartment on Northwest 61st Street,
mother, three sons, two daughters and a grandson.  A plastic table, two chairs
with rusted seats, a worn sofa covered with a tattered sheet and a television
set are all that is in the living room-kitchen area. Roaches scurry back and
forth across the walls.

The mother, Clary Lee Capers, a small woman whose soft features reflected her
anguish, said the family was down on its luck, with no hopes for a better
future.

''But we're not murderers,'' she said.  Mrs. Capers said her sons had been out
earlier but were at home with her, watching television, when the trouble began a
block from their home. Moving to the kitchen window that overlooks the parking
lot where Benny Higdon and his friends were killed, Mrs. Capers said she and her
family had watched from there as the mob vented its anger.

Thought It Was a Wedding

''We kept hearing cars blowing their horns and at first we thought that maybe it
was a wedding,'' she said.  ''The first we knew that they thought my sons had
did it was about 6:30 Monday morning when the police busted in with guns,'' she
said.  ''I know they didn't do it,'' Mrs. Capers insisted, on the verge of
tears. ''They were here with me. I know they didn't do it.'' Both Leonard and
Lawrence Capers dropped out of school; both have police records. Leonard had
been arrested for theft and driving without a license. There was a warrant
outstanding for him on the license charge at the time of the riots. The records
did not show the disposition of the theft charge.

Lawrence's record includes arrests for larceny, assault and battery, possession
of stolen property, carrying a concealed weapon, possession of drugs and
inhaling a chemical substance. He was fined in three cases, sentenced to jail in
two others and given suspended sentences in still others.

On the murder charges the Caperses and Mr. Lane are represented by the public
defender. They have not been indicted. The prosecution says it will ask that the
two juveniles arrested with them be tried as adults.

Some Make It; Some Don't

The riot was an exaggeration of everyday life in Liberty City, according to the
man who was there at the start of the beating of the Kult brothers.

''That's the way it be on the street,'' he said. ''Some make it and stay out of
trouble and others don't. Some people get jobs and other people don't.

''Some dudes just don't care,'' he continued. ''If you have it and they want it
they take it if they can. Don't make no difference if you're black or white.''

LANGUAGE: ENGLISH

GRAPHIC: Illustrations: Photo of store damaged by looters

                   Copyright 1980 The New York Times Company


                                3 of 8 DOCUMENTS

                               The New York Times

                 June 2, 1980, Monday, Late City Final Edition

CONSERVATIVE AND ANTIABORTION GROUPS PRESS ATTACK AGAINST McGOVERN

BYLINE: By LESLIE BENNETTS, Special to the New York Times

SECTION: Section B; Page 11, Column 1; National Desk

LENGTH: 1318 words

DATELINE: SIOUX FALLS, S.D.


Churchgoers across South Dakota returned to their cars yesterday to find
brochures featuring pictures of a dead fetus and of Senator George McGovern's
face slashed with a large X, along with the assertion that he ''has continuously
voted tax dollars to kill preborn children.''

Until tomorrow, when Mr. McGovern faces his first Democratic primary challenge
in his 18 years in the Senate, all South Dakotans, churchgoers and
nonchurchgoers alike, will be unable to listen to their radios for long without
hearing commercials denouncing the Senator as a big-spending liberal who is weak
on national defense.

By now such arguments are almost as familiar to most South Dakotans as the lined
face of their senior Senator, the 1972 Democratic Presidential nominee. Unlike
the wheat, cattle and hogs on which this state's economy depends, however, the
negative advertising is not entirely homegrown. Nor will it cease if Mr.
McGovern defeats his relatively unknown opponent, Larry Schumaker, as he is
expected to do, and again becomes the Senate nominee of the party he began
building here almost single-handedly 25 years ago.

A Top-Priority Target

For almost two years the mailings and leaflets and the television, radio and
newspaper advertisements have been pumped into South Dakota by national
anti-abortion and conservative political groups that have joined forces to try
to defeat liberal legislators around the country. Senators Frank Church of
Idaho, Birch Bayh of Indiana, John C. Culver of Iowa and Alan Cranston of
California are also on the socalled ''hit lists'' of many of these groups, but
Senator McGovern is their top-priority target.

''McGovern is a liberal Senator from a conservative state, and his record really
is outrageous,'' said John T. Dolan, executive director of the National
Conservative Political Action Committee, a Virginia-based organization that
sponsored the current radio commercials. ''Not every member of the U.S. Senate
pals around with Castro. We felt we could make McGovern vulnerable.''

Mr. McGovern says the assault has taken its toll. ''It's hurt,'' he remarked.
''I'm always answering negative questions: 'Is it true you're a pal of Fidel
Castro's, that you sold the Panama Canal to the Communists, that you want to
kill the unborn?' It's all these offthe-wall right-wing ideas. They've been
running this stuff for two years, and there's no question it creates a negative
mood.''

Setting Him Up for November

While few of his critics believe Mr. McGovern can be defeated in the primary,
they hope the damage inflicted will help set him up for defeat in November.

The National Conservative Political Action Committee alone has spent an
estimated $120,000 to attack Mr. McGovern and promote his rivals, chief among
them the leading Republican candidate, Representative James Abdnor.

''If we had $2 million, we'd spend it against him,'' Mr. Dolan said, adding that
the $3.4 million that the committee has already spent on campaigns around the
country will rise to $5.5 million by November.

Among the other national organizations opposing Mr. McGovern are the National
Right-to-Life Committee, Life Amendment Political Action Committee, Committee
for the Survival of a Free Congress, National Conservative Caucus, American
Conservative Union, Fund for a Conservative Majority, Young Americans for
Freedom, the John Birch Society, Citizens for the Republic, National Right to
Work, Eagle Forum, Citizen's Committee for the Right to Keep and Bear Arms, Gun
Owners of America, Tax Limitation Committee, Committee to Defeat the Union
Bosses and the Committee to Save the Panama Canal.

South Dakota has spawned its own set, including People for an Alternative to
McGovern, the Senatorial Research Committee, People Who Want a New Senator, and
Target McGovern, an organization started by Dale Bell, another contender for the
Republican Senate nomination and a campaign aide to Ronald Reagan in 1976.

Target McGovern has sent out more than 200,000 pieces of literature in the last
year, or enough to reach almost a third of the state's population.

'A Glaring Loophole'

Mr. McGovern estimates these groups have spent more than $250,000 to attack him
and promote his rivals.  Such expenditures are made possible by what the Senator
calls ''a glaring loophole'' in Federal election laws. No group or individual
may contribute more than $5,000 to support a given candidate, but unlimited
funds may be spent against a candidate, or on ''independent'' efforts in favor
of a candidate, as the National Conservative Political Action Committee is doing
in support of Mr.  Abdnor, the Republican front-runner.

While Mr. Abdnor concedes that a poll the committee conducted and brought to him
last fall influenced his decision to seek the nomination, he denies that his
announcement was coordinated with a flood of anti-McGovern television
advertisements that appeared immediately before and after he entered the race.
The McGovern forces, charging collusion, have filed a complaint with the Federal
Election Commission; last week Mr. Dolan filed a countercomplaint, contending
that Mr. McGovern was using the F.E.C. as a ''political football.''

Worried About an Alliance

The Senator, who has already spent more than $750,000 on his reelection effort,
says he is troubled by the emerging alliance between anti-abortion and
conservative political interests. ''I worry about the right-to-lifers getting
into bed with these extremist right-wing groups,'' he said. ''A lot of
right-to-lifers are being used by the right-wing without realizing they're sort
of political dupes.''

In his view, the anti-abortion groups are responsible for the candidacy of his
opponent in the primary, Mr. Schumaker, a mathematics teacher from Texas who
supports a constitutional amendment outlawing abortion. Mr. Schumaker, a
political novice, was born in South Dakota but left 1961 and did not return
until he announced his candidacy last winter.

''I think the right-to-lifers brought Larry Schumaker up here from Texas and are
providing the organizational strength, the bodies, everything,'' said Mr.
McGovern, whose votes in Congress have supported legislation that would let
women make their own choices regarding abortion. The National Abortion Rights
Action League contributed $4,900 to his campaign.

Mr. Schumaker has been endorsed by the South Dakota Life Amendment Committee and
his campaign is being promoted by national antiabortion groups like the
Virginia-based Life Amendment Political Action Committee, which printed the
100,000 brochures for distribution yesterday in church parking lots.

Supported by Religious Leaders

According to some South Dakotans, the anti-abortion drive against Sen. McGovern
has long received the tacit, and sometimes active, support of many religious
leaders, such as the Rev. Thomas Burns of St. John de Britto Church in Britton,
who recently wrote fellow Roman Catholic priests urging them to oppose Mr.
McGovern and support Mr.  Schumaker.

To encourage such political involvement, the Life Amendment Political Action
Committee recently mailed 800 lists of ''do's and don'ts of political activity
for pastors.'' Paul Brown, the Life Amendment Political Action Committee's
director, asserted: ''A pastor can get up on the pulpit and say he has
personally endorsed a candidate if he makes it clear he's not giving the
church's endorsement.''

LANGUAGE: ENGLISH

GRAPHIC: Illustrations: photo of Senator McGovern

                   Copyright 1980 The New York Times Company


                                4 of 8 DOCUMENTS

                               The New York Times

                 June 2, 1980, Monday, Late City Final Edition

CUBANS RIOT AT CAMP IN ARKANSAS;
POLICE, REFUGEES AND CIVILIAN HURT

BYLINE: By The Associated Press

SECTION: Section A; Page 1, Column 4; National Desk

LENGTH: 1152 words

DATELINE: FORT CHAFFEE, Ark., June 1


Rioting broke out in the Fort Chaffee refugee relocation center this evening as
about 200 Cubans, angered by delays in the processing of their cases, pelted the
police and soldiers with stones after bursting out the front gate. Between 15
and 20 law enforcement officers, at least one civilian and four refugees were
hurt before order was restored, the authorities said.

The refugees retreated into the sprawling Army reserve base after state troopers
fired about 20 shots from pistols and shotguns.  Three refugees were admitted to
the St. Edward Medical Center near the base with gunshot wounds, according to
Jim Hannah, a hospital spokesman. A fourth refugee was admitted with a stab
wound. One civilian who was hit by a stone was treated at the emergency room,
Mr. Hannah said, and medical personnel treated several other persons at the
base.

Official Denies Takeover Reports

Federal police officers and soldiers from Fort Sill, Okla., used tear gas and
clubs in an attempt to subdue the refugees once they were back inside the base.
The Army Reserve base, near the city of Fort Smith in northwestern Arkansas,
contains a resettlement camp now occupied by 18,000 refugees awaiting
relocation.

Maj. Brian McWilliams said late tonight that the rioting had been quelled and
order restored.  Earlier, Bill McAda of the Federal Emergency Management Agency
denied reports that the refugees had taken over the base, but said that bands of
refugees were milling about inside.

Fire Is Set in Street

About 10 minutes after the refugees retreated inside, they piled small wooden
guardhouses and barricade sawhorses in a street on the base and set them afire.
There also was a report of fires in some barracks. A base fire truck was called
out.

Some Cubans helped to put out fires and quell the rioters. Major McWilliams said
a number of Cubans had been arrested, but he did not know how many.

Gov. Bill Clinton placed the Arkansas National Guard on alert and asked the
White House for more military assistance to control the camp. President Carter
dispatched Eugene Eidenberg, a White House staff member, and Tom Casey of the
Federal Emergency Management Agency to the scene tonight.

More than 200 civilians were evacuated from the base, Mr. McAda said.  The
incident was the second of the day in which refugees burst out of the base. Army
troops and police wielding nightsticks turned back about 300 refugees who
escaped earlier this afternoon.

Four Policemen Hurt

The first breakout, at about 1 P.M., followed a stone- and bottlethrowing
incident that left three military policemen injured.  Four other law enforcement
officers were hurt last night in a similar incident when refugees hurled stones
and bricks.

Fifty to 100 of the escaping refugees got to within 50 yards of the Barling city
limits, less than a mile from Fort Chaffee, before they were stopped. All were
back inside the camp by midafternoon.

The refugees in the first breakout vaulted over a low wall at the compound. Some
200 of them were stopped by a cordon of state troopers on Arkansas Highway 22
between the fort and the city of Barling, about a mile away. The others were
caught by armed soldiers at a roadblock just outside Barling. The pursuing
officers reportedly struck a number of refugees on their heads, backs and legs.

One Cuban was taken to a hospital for treatment of a head injury.  The
authorities said that another Cuban required treatment of a wound in the abdomen
after he was apparently was stabbed by a fellow refugee.

The authorities at the base said that about 150 Cubans began throwing stones and
bottles around noon today.  One military policeman suffered a broken foot.
Injuries to the other two were said to be minor.

Angered by Delays in Processing

The refugees, frustrated by delays in processing, shouted ''Libertad'' and
marched down the camp's main street toward the front gate. The mob swelled to
300 as refugees at the gate and some along the way joined the march.

About a dozen military policemen in riot gear at the gate made no attempt to
stop the Cubans from leaving.  Brig. Gen. James E. Drummond, commander of the
military force, attempted to reassure the Cubans that their processing was being
handled as quickly as possible.

The attempted escapes and the two stone-throwing incidents were part of a series
of confrontations between law enforcement officials and refugees. The base has
been the scene of numerous demonstrations in the last week, with as many as
1,000 refugees from the Cuban flotilla gathering to demand their freedom.
Protests have also been conducted outside the base by Cuban-Americans and local
residents.

Processing Center Closed

As a result of last night's disturbance, officials of the immigration service
closed down their processing center temporarily, delaying the release of about
300 Cuban refugees who had relatives waiting.

The officials announced yesterday that they would speed up the processing of all
refugees who had relatives waiting. However, only 11 of the 300 refugees were
released before the office was closed.

The office was reopened this morning, but was shut down periodically while the
demonstrations were going on, on orders of the Federal Emergency Management
Agency.

----

Governor Complains of Security

Special to the New York Times

FORT CHAFFEE, Ark., June 1 - Gov. Bill Clinton complained tonight that he ''may
have been misled'' by Federal authorities last week when they said they would
use force, if necessary, to contain any further disruptions by the Cuban
refugees at the processing center here.

At a news conference the Governor said state and local forces had provided the
only security in the two disturbances today.  ''There's a real authority problem
here,'' he said. ''Who's in charge? I think it's being run by a committee.'' He
said 1,700 National Guard and Army troops were on duty or on alert outside the
fort or within 90 miles of it.  The Governor also ordered sheriff's and state
police units to use loudspeakers to inform residents of Sebastian County that
the Cubans had been returned to the base.

Of the reugees, he said that ''some may be agents'' of Fidel Castro ''or else
they do not understand freedom.'' And he declared: ''I am tired of seeing our
people upset and frightened.''

----

Cuban Influx Virtually Halted

KEY WEST, Fla., June 1 (AP) - The Cuban refugee flotilla was at a virtual
standstill today, with only one vessel arriving in Key West.  American officials
said that they believed it was only a temporary halt in the sporadic operation
before an expected final surge in the next week or two.

The single boat to arrive by midday today carried 128 refugees, bringing the
total to 94,269 since the crossings began in April.

LANGUAGE: ENGLISH

GRAPHIC: Illustrations: PHOTO OF CUBAN REFUGEES AT FORT CHAFFEE ARK.

                   Copyright 1980 The New York Times Company


                                5 of 8 DOCUMENTS

                               The New York Times

                 June 1, 1980, Sunday, Late City Final Edition

HARASSMENT IS FOCUS OF RIFT AT WESLEYAN

BYLINE: By DAN COLLINS

SECTION: Section 11; Connecticut; Page 6, Column 5; Connecticut Weekly Desk

LENGTH: 1277 words


MIDDLETOWN LAST Apr. 4, a 20-year-old woman student at Wesleyan University
walked into a campus fraternity house at 4 A.M. and was confronted and then
grabbed by two fraternity men, who picked her up, spun her around and kissed
her. Badly frightened, she ran from the house.

A decade ago such an episode might have passed without much notice or comment,
but the incident at the Chi Psi fraternity house has aroused a bitter
controversy at the small liberal arts college.

The Wesleyan faculty has requested the formation of a special committee to
examine the problem of sexual harassment at the school and a women's group has
accused Colin G. Campbell, the president of the university, of taking action
that will discourage female students from reporting incidents of sexual
harassment or abuse.

Much of the acrimony centers on Mr. Campbell's decision to reduce the penalties
meted out to the two fraternity men by a student court, but the debate is being
carried on in an atmosphere in which the safety of women students on campus has
become a widespread concern.  Eight incidents of sexual assault were reported on
or near campus this school year and last month, Argus, the student newspaper,
devoted most of the front page of an issue to stories about the episode
involving the fraternity men, a rape near campus and the transfer of a security
guard accused of harassing women students.

The current conflict also reflects a sharp clash in values between women's
groups on campus and a male fraternity that has, according to several of its own
members, a reputation as an ''animal house.''

''We're thought of as animals and there isn't much sympathy for us here on
campus,'' a member of Chi Psi said. ''A lot of the kids are very serious about
their grades and so forth but we like to play football, party and do a lot of
social things.''

Wesleyan has a high academic reputation and takes pride in the ability and
diversity of its students. The previously all-male college went coeducational 10
years ago and now about half of its 2,500 students are women. The
once-predominant role of fraternities on campus has declined sharply in the last
decade and today only about 10 percent of the students are members, according to
school officials. At the same time, feminist issues and questions about the role
and status of women have become prominent topics at Wesleyan, as they have on
other campuses.

The woman who was allegedly accosted by the two fraternity men filed a formal
complaint against them with a college official, who referred the matter to the
Student Judicial Board, a five-member student court that has the power to expel
students for disciplinary infractions. After a formal hearing last month, the
court decided to suspend one of the men, a junior, from school for one semester
and to withhold the diploma of the second student, a senior, for one year.

The two students appealed the court's decision to Mr. Campbell, and last week,
for only the second time in his 10 years as president at Wesleyan, Mr. Campbell
reduced the penalties imposed by the student court.

The case was the first in which Wesleyan students had been disciplined as a
result of allegations of sexual harassment. In a prepared statement, Mr.
Campbell said that the court had established a new ''standard of conduct'' for a
college community whose values were ''undergoing rapid change.''

The president also said that Wesleyan would not condone such conduct and that
''notice has been given that such violations will be dealt with severely.'' He
said that his decision to reduce the penalties in the case of the two fraternity
men ''should not dull either our awareness of that fact or the educational
purpose which has been served.''

Mr. Campbell said that he approved of the new standard established by the board,
''but in this case, the violators of that standard did not have the advantage of
prior notice and therefore could not be expected to anticipate such severe
penalties as a consequence of their actions.''

One of the fraternity men, the junior, was placed on disciplinary probation. The
senior was assigned to do an unidentified ''project.'' (The project is writing a
30-page paper on sexism.)

Mr. Campbell's decision to reduce the penalties met with bitter opposition from
a coalition of women's groups, who asked him to reverse his decision. Mr.
Campbell refused.

''The administration's action can be interpreted as meaning that sexual assault
and harassment do not exist because we don't have any rules about this kind of
behavior,'' said Jane Cooper, a senior, who is active in an ad hoc student
committee on sexual assault and harassment. Wesleyan has no disciplinary rules
that deal with sexual harassment and one of the changes women's groups on campus
are seeking is a specific grievance procedure to handle such complaints.

Some faculty members are also sympathetic. Last week, the faculty approved a
resolution requesting the establishment of a committee that will investigate the
problem and explore the feasibility of establishing a grievance procedure for
complaints involving sexual harassment.

''Women basically do not feel safe walking across campus or walking into a
fraternity house,'' said Allan Johnson, an assistant professor of sociology.
''The administration really doesn't know what to do about this problem and it
isn't particularly motivated to do anything about it.''

Mr. Campbell could not be reached for comment but a spokesman for the university
said that the president was concerned about the problem and that the
administration would be receptive to any suggestions from faculty, staff or
students.

The debate about Mr. Campbell's decision has also been marked by confusion since
school officials have declined to identify the woman complainant or the two
fraternity men. The woman could not be reached for comment but the fraternity
men agreed to be interviewed on the condition that their names not be used.

''We were just fooling around. Playing. It was nothing. We picked her up, swung
her around in the air and kissed her. She became hysterical and ran out,'' one
of the two students said.

''This happens to us for kissing a girl at 4 o'clock in the morning. It's
unbelievable,'' the other fraternity man said. (The woman, who was looking for
her boyfriend, had walked in on an early morning party at the fraternity house
when the incident occurred.)

Both men denied that the incident had any sexual overtones and both accused
their female accuser of exaggerating the episode completely out of proportion.

The woman alleged that the two men pinned her to a wall while one of them thrust
his tongue into her mouth. Both students denied this but one of them admitted
that he had not been entirely truthful with the student court. ''I lied to the
board. I told them I'd had about five beers when I really had about 15,'' he
said.

The student court, however, ruled that the incident was not harmless horseplay
and it has not changed its mind, according to Julia Imber, a senior who has been
on the board for two years. ''We heard both sides and reached our own
conclusions,'' she said. ''We don't dispute the right of the two students to
appeal the matter to President Campbell but we stand by our original decision.''

LANGUAGE: ENGLISH

                   Copyright 1980 The New York Times Company


                                6 of 8 DOCUMENTS

                               The New York Times

                 June 1, 1980, Sunday, Late City Final Edition

700 Motorcyclists Press Police to Act on Fatality

BYLINE: AP

SECTION: Section 1; Part 1; Page 33, Column 1; Metropolitan Desk

LENGTH: 200 words

DATELINE: BRISTOL, Conn., May 31


More than 700 motorcycles lined the street outside Police Headquarters here
today as cyclists demanded action against a state legislative leader who was in
an accident last January in which a motorcyclist was killed.

The four-hour demonstration was generally peaceful. It ended as the cyclists,
many riding double and drinking beer, roared out of the city with a police
escort in front of an estimated 5,000 spectators.  No arrests or major injuries
were reported.

Two organizers of the protest met with Police Chief John Oliver to demand that
State Representative Andrew R. Grande, a Bristol Democrat who is co-chairman of
the General Assembly's General Law Committee, be charged in connection with the
accident on Jan. 16. Thomas Swingle, 24 years old, of Bristol, was killed in the
crash when the motorcycle he was operating on Middle Street here collided with a
car driven by Mr. Grande.

Police officers who investigated the accident said Mr. Grande was at fault, but
the assistant state's attorney, Patrick McGinley, said prosecution was not
warranted. Mr. Grande could not be reached for comment today.

LANGUAGE: ENGLISH

                   Copyright 1980 The New York Times Company


                                7 of 8 DOCUMENTS

                               The New York Times

                 June 2, 1980, Monday, Late City Final Edition

BESIEGED MOHAWK FACTION RESISTS TRIBAL MAJORITY

BYLINE: Special to the New York Times

SECTION: Section B; Page 3, Column 1; Metropolitan Desk

LENGTH: 999 words

DATELINE: ST. REGIS RESERVATION, N.Y.


''It's the longest siege in Indian history,'' said Howard Berman, a lawyer, as
he walked through the traditionalist Mohawk encampment near Racquette Point on
the St.  Lawrence River, about eight miles northeast of Massena.

Since mid-August 1979, Mr. Berman's clients, a small group of traditional - or
longhouse - Mohawks, have lived within a fourbuilding compound in sight of the
international bridge that crosses the St. Lawrence from the United States to
Cornwall, Ontario. Although the ''besieged'' have not been forced to leave, they
have stationed sentries to warn them in case state or tribal authorities try to
enter, which they did once last year, making several arrests.

At least 15 of the perhaps 50 Mohawks staying here face state charges stemming
from a nine-hour takeover of the Akwesasne Community Center on the reservation
in reaction to the arrest of a chief over a property dispute. The action, on May
29, 1979, pitted traditionalist Mohawks against others who support the
reservation's elective system of government. The takeover ended peacefully.

The anniversary of the takeover passed last Thursday, but the basic
jurisdictional issues that have divided this reservation for decades remain
unsolved. Lately, however, the traditionalists have come to believe that their
resistance has won them supporters not only outside, but also inside, the
reservation.

Traditionalist Is Optimistic

The traditionalists assert that the path they are following leads back to an
agriculture-based economy and away from an economy based on government programs,
which has developed in the last 20 years. The movement is parallel to
developments on other reservations throughout the country. At the same time, two
other rival groups on the reservation are vying for power to control millions of
dollars in grants.

''Nine months ago we got 108 people to sign a petition asking that charges be
dropped against the occupiers,'' said Mike Myers, a traditionalist. ''Recently,
we got 1,035 signatures on the same petitions. People have closed together in a
resistant stance. They are saying enough is enough. The system has to change.''

The St. Regis reservation has a crazy-quilt of governmental systems, with
overlapping county, state and national codes, as well as competing tribal
authorities.

''It takes years to understand this place,'' said Diane Lazore, who was born on
the lands of the Canadian St. Regis Band, which bordered those of the Mohawks of
the St. Regis reservation in the United States. Each reservation has a separate
government, but Mohawks pass freely between both sides. There are approximately
3,000 Mohawks on the American side and 7,000 in Canada.

Elective System of Government

There has been an elective system of government on the reservation in the United
States since the late 1880's. Three chiefs are elected, one each year, for
three-year terms. The election is traditionally held in June, although there is
no constitution or formal elective procedure.

The traditionalists have their own system of hereditary chiefs, and they contend
that these chiefs should govern the entire reservation.  Central to the
traditionalist position is the contention that the Mohawks of St. Regis are one
nation within the six-nation Iroquois Confederacy. As members of a sovereign
nation, the traditionalists do not recognize state or Federal authorities. The
elective system's supporters make no claim for nation status or sovereignty.

Traditionalists refuse to vote in tribal elections, and Rudolph Hart Jr., the
tribal planner and a son of one of the elective chiefs, believes that this
refusal to vote makes the traditionalists an insignificant factor in tribal
politics.

'Dissident' Faction Emerges

Mohawk politics were further confused or clarified - it depends upon whom one
talks to - by the emergence this spring of a third faction, the so-called
''dissident'' party, which held an election and replaced the three tribal
chiefs. But the State of New York has not recognized the election, which was not
the traditional June voting, and the previously elected chiefs remain in office.
The traditionalists have supported the dissidents.

Mr. Hart sees the emergence of the dissidents as a simple attempt to gain
political power. ''In 1969, the Mohawks received a total of $2,200 in outside
grants,'' he said. ''Last year, we received over $5 million. The stakes here are
control of that $5 million.''

Also at stake, according to Mr. Hart, is the settlement of a Mohawk land claim
suit. In exchange for their surrender of a claim upon land that includes
downtown Massena, the elected chiefs have tentatively agreed to accept $6
million and 9,750 acres of state forest land. Dissidents and traditionalists
oppose the accord.

Debate over the land claim settlement is recent, but a controversy over the size
and power of the tribal police force is long-standing.  An arm of the Franklin
County Sheriff's Department, the force is composed entirely of Indians.
Recently, a Federal training grant has allowed the tribe to increase the number
of police from 7 to 17.

Mr. Berman, a non-Indian who along with William M. Kunstler, has been
representing the traditionalists in court, sees the expansion of the police
force as ''either a dangerous move toward the creation of a police state or a
dangerous abuse of welfare funds.''

On June 20, the Mohawks arrested for the community center occupation are to go
to trial on conspiracy charges. Loran Thompson, the traditionalist chief whose
arrest in May 1979 led to the takeover but who was not indicted for the
occupation, sees mere participation in the trial as a defeat. ''We may win in
court,'' he said. ''But just by being there we lose some of our sovereignty.''

LANGUAGE: ENGLISH

GRAPHIC: Illustrations: PHOTO OF LORAN THOMPSON

                   Copyright 1980 The New York Times Company


                                8 of 8 DOCUMENTS

                               The New York Times

                 June 2, 1980, Monday, Late City Final Edition

DETAILS RELEASED ON PLAN TO SAVE TUDOR CITY PARKS

BYLINE: By CARTER B. HORSLEY

SECTION: Section B; Page 1, Column 1; Metropolitan Desk

LENGTH: 783 words


Details of an agreement that calls for an indefinite delay of the demolition of
two parks at Tudor City were disclosed yesterday. Both sides said they would
seek court approval of the agreement today.

The two parks straddle 42d Street west of First Avenue. Under the agreement, the
delay would permit the completion of plans for swapping the two parks for a
city-owned playground on the southeast corner of 42d Street and First Avenue.

The Tudor City parks are owned by companies headed by Harry B.  Helmsley. If the
swap can be effected, Mr. Helmsley would build an apartment and office building
of about 50 stories on the site of the city playground.

The agreement would allow the plan for the exchange to be worked out by Mr.
Helmsley, city and state officials and the Tudor City Tenants Association.

New Park Facility Possible

According to State Senator Roy M. Goodman - who held a meeting Friday of people
involved in the 10-year-old effort to save the parks - the solution might call
for Mr. Helmsley to contribute toward the another community recreational
facility to replace the playground involved in the swap.

One site under serious consideration, he said, is a city-owned parking lot
between 34th and 35th Streets on a strip along the East River, east of the the
Franklin D. Roosevelt Drive.

Harry Scanlan, construction and development director for Helmsley Enterprises,
said that if the 34th Street playground proposal was ''in the realm of reason,
Mr. Helmsley will give it consideration.''

Addressing a rally of residents yesterday afternoon in the parks, Senator
Goodman, Republican of Manhattan, said that a wooden fence around the north park
would be taken down tomorrow morning.  Construction workers had started putting
up the fence early one morning over the Memorial Day weekend, but the work was
quickly halted by residents who first carried off the lumber and then obtained a
restraining order that is answerable today in State Supreme Court.

Court's Approval Sought

Rather than hearing further arguments on the restraining order, the court will
be asked to accept the new agreement, which was reached Friday.

Under the agreement, Mr. Helmsley would give the tenants' association 15 days'
notice of any plan to resume demolition. In return, the restraining order is to
be lifted and Mr. Helmsley is to drop his motion to dismiss it.

The agreement was contingent on having the State Department of Parks put off a
June 13 hearing on a proposal that the Tudor City complex be recommended by the
state to be placed on the National Register of Historic Places. And the
department did agree over the weekend to defer the hearing, without setting a
new date. The historic designation would have posed a procedural hurdle for Mr.
Helmsley.

The Parks Department will give 30 days' written notice of a rescheduling of a
hearing on Tudor City.  The plan for a swap was first put forward more than a
year ago by the city, in response to a proposal by Mr. Helmsley to erect a
50-story residential tower on a small city-owned park opposite the United
Nations Secretariat.

Helmsley Approved Swap

The city's proposal was acceptable to Mr. Helmsley, and the State Legislature
passed bills introduced by Senator Goodman and Assemblyman Steven Sanders
permitting the city to make the swap.

In an interview yesterday, Mr. Scanlan of Helmsley Enterprises said that much of
the project's difficulty stemmed from conflicting appraisals of the city-owned
parcel at 42d Street and First Avenue.  Last winter, he said, one of three
appraisers required, under state law, for any such exchange of land, disagreed
with the basis for appraisal. The appraiser, whose name was not made public,
said the parcel should be appraised at its highest and best use, namely for a
hotel, rather than a residential tower.

It could not be learned what the valuation total was, but, according to Mr.
Scanlan, the dissenting appraiser's valuation of the site was $100,000 or so
more than the other two. Under state law, the city must get property equal in
value to the property it is giving up, and the exchange may not involve money.

Sentiment on Planning Unit

Mr. Goodman said the City Planning Commission was hopeful that the dissenting
appraiser would revise his valuation once a deed was drawn up for the playground
site restricting any new building mostly to apartments, with some office space,
and prohibiting a hotel.

LANGUAGE: ENGLISH

                   Copyright 1980 The New York Times Company"""

In [17]:
# with open('/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_1/row_0.txt', encoding='utf8') as r:
#     text = r.read().replace('\n', ' ').replace('\r', '')
prompt = f"""Task:Given the text {text}, generate a summary"""
delimited_text = ollama.generate(model="mistral", prompt=prompt)["response"]
delimited_text


" An agreement has been reached to delay the demolition of two parks at Tudor City, owned by Harry B. Helmsley's companies. The delay will allow for the completion of plans to swap these parks for a city-owned playground on 42nd Street and First Avenue. If successful, Helmsley would build an apartment and office building of about 50 stories on the site of the city playground. The agreement allows for work to be done by Helmsley, city, state officials, and the Tudor City Tenants Association to determine if a new community recreational facility will replace the playground involved in the swap. One possible location is a city-owned parking lot between 34th and 35th Streets along the East River. Construction workers had started putting up a fence around one of the parks over the Memorial Day weekend, but it was quickly halted by residents who obtained a restraining order. The agreement is contingent on the State Department of Parks deferring a June 13 hearing on Tudor City's historic desig

In [18]:
# with open('/home/niamatzawad/niamatzawad/Datasets/HotpotQA/hotpot_1/row_0.txt', encoding='utf8') as r:
#     text = r.read().replace('\n', ' ').replace('\r', '')
prompt = f"""Task:Given the text {text}, generate some labels which best classify the theme of the text. 

             ###Example of an answer:
             1. Real Estate Development\n2. City Planning\n3. Legal Proceedings\n4. Community Activism\n5. Historic Preservation\n6. Politics (State and Local)\n7. Parks and Recreation\n8. Business and Finance\n9. Urban Development"""
labels = ollama.generate(model="mistral", prompt=prompt)["response"]
pattern = r'\d+\.\s*(.+)'
match = re.findall(pattern, labels)
print(match)

['Real Estate Development', 'City Planning', 'Legal Proceedings', 'Community Activism', 'Historic Preservation', 'Politics (State and Local)', 'Parks and Recreation', 'Business and Finance', 'Urban Development']


In [19]:
# %%time
# folder_path = "/home/niamatzawad/niamatzawad/Datasets/UTDBox/demoNY/unsplitted"
folder_path = "hotpot_1"

s_demo = SemanticSimilarity()
kwe_demo = KeywordExtractor()
kg_demo = KG()

folder_objects = os.listdir(folder_path)
for i, filename in enumerate(folder_objects):
    if os.path.isfile(os.path.join(folder_path, filename)):
        with open(os.path.join(folder_path, filename), 'r') as file:
            text = file.read()
            text_splitted = [text[i * 1900:(i+1) *1900] for i in range(len(text)//1900)]
            start_text_time = time.time()
            for text_s in tqdm(text_splitted):
                start_subtext_time = time.time()
                try:
                        named_entities = kwe_demo.extract_named_entities(text_s)
                except Exception as e:
                    continue
                s_demo.generate_reference(named_entities)
                t = TripletGenerator(text_s, s_demo, kwe_demo, os.path.join(folder_path, filename), named_entities=named_entities)
                triplets = t.get_triplets()
                kg_demo.add_triplets(triplets)
                print(f"Time to create triplets for current subtext - {time.time() - start_subtext_time}")
                
        print(f"Finished processing {i+1}/ {len(folder_objects)} files")
        print(f"Total time taken for current text - {time.time() - start_text_time}")
        print("---")
        


  0%|                                                                                            | 0/2 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
C:\Users\zawad\miniconda3\envs\phd\Lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Starting server with command: java -Xmx8G -cp C:\Users\zawad\.stanfordnlp_resources\stanford-corenlp-4.5.3/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-8a583bb2dc6f4b12.props -preload openie
Corpus: Radio City is India's first private FM radio station and was started on 3 July 2 [...].
Found 121 triples in the corpus.
[...]
the text is 
Radio City is India's first private FM radio station and was started on 3 July 2001.  It broadcasts on 91.1 (earlier 91.0 in most cities) megahertz from Mumbai (where it was started in 2004), Bengaluru (started first in 2001), Lucknow and New Delhi (since 2003).  It plays Hindi, English and regional songs.  It was launched in Hyderabad in March 2006, in Chennai on 7 July 2006 and in Visakhapatnam October 2007.  Radio City recently forayed into New Media in May 2008 with the launch of a music portal - PlanetRadiocity.com that offers music related 

 50%|██████████████████████████████████████████                                          | 1/2 [00:11<00:11, 11.06s/it]

The matches are
[]
the triplets are
[('India', 'has', 'first private FM radio station'), ('Radio City', 'is', "India 's FM radio station"), ('Radio City', 'was', 'started'), ('Radio City', 'is', "India 's first FM radio station"), ('Radio City', 'was started on', '3 July 2001'), ('Radio City', 'is', "India 's first private FM radio station"), ('Radio City', 'is', "India 's private FM radio station"), ('2003', 'since 91.1 is', 'Lucknow'), ('2003', 'since 91.1 is', 'Bengaluru'), ('It', 'broadcasts on', 'Bengaluru'), ('It', 'plays', 'Hindi'), ('It', 'was', 'launched'), ('It', 'was', 'launched in Chennai on 7 July 2006'), ('It', 'was launched in', 'Visakhapatnam October 2007'), ('It', 'was', 'launched in Hyderabad in March 2006 on 7 July 2006'), ('It', 'was', 'launched on 7 July 2006 in Visakhapatnam October 2007'), ('It', 'was', 'launched in March 2006 in Chennai on 7 July 2006 in Visakhapatnam October 2007'), ('It', 'was', 'launched in Hyderabad in March 2006 in Chennai in Visakhapatnam 

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:17<00:00,  8.97s/it]

The matches are
['Music: Artist Achievements and Records', "Education: Women's Colleges in the Southern United States", "History: Smallest Courthouse Museum in the U.S., Arthur's Magazine", 'Sports: Ukrainian Hockey Championship', 'Magazine: First for Women (women-focused publication)']
the triplets are
[('ARIA', 'is in', 'Australia'), ('es', 'is in', 'United States'), ('song', 'was', 'Warner Bros.'), ("Records ' digital song", 'is with', '1.3 million downloads sold'), ('Records', "'", 'digital song of 2014 with 1.3 million downloads sold'), ("band 's debut album", 'was', 'released'), ('band', 'has', 'debut album'), ("band 's debut album", 'was released on', 'October 8 2013'), ('student populations', 'consist exclusively or almost exclusively of', 'women located'), ('bachelor', 'has', 'degreeâ'), ('student populations', 'consist exclusively or exclusively of', 'women'), ('student populations', 'consist exclusively or almost exclusively of', 'women located in Southern United States'), (

In [21]:
def get_subjects_from_triplets(triplets):
    return list(set([i[0] for i in triplets]))
get_subjects_from_triplets(kg_demo.triplets)

['Albania',
 'nArthur',
 'nFootball',
 ' Radio City is India\'s first private FM radio station, launched on July 3, 2001. It broadcasts in Hindi, English, and regional languages across various cities including Mumbai, Bengaluru, Lucknow, New Delhi, Hyderabad, Chennai, and Visakhapatnam. The station began operations in multiple cities at different times, with the first broadcast starting in Bengaluru in 2001, followed by Mumbai in 2004. Abraham Thomas is the CEO of the company.\n\nFootball in Albania predates the Albanian Football Federation (FSHF), as evidenced by the team\'s registration at the Balkan Cup tournament during 1929-1931. Despite facing pressure from other teams due to competition, football activities started first and were robust enough for duels. The Albanian National Team was founded on June 6, 1930, but had to wait until 1946 to play its first international match against Yugoslavia, which they won. Albania joined FIFA in 1932 during the 12th-16th June convention and wa

In [22]:
set(kg_demo.triplets)

{(' Radio City is India\'s first private FM radio station, launched on July 3, 2001. It broadcasts in Hindi, English, and regional languages across various cities including Mumbai, Bengaluru, Lucknow, New Delhi, Hyderabad, Chennai, and Visakhapatnam. The station began operations in multiple cities at different times, with the first broadcast starting in Bengaluru in 2001, followed by Mumbai in 2004. Abraham Thomas is the CEO of the company.\n\nFootball in Albania predates the Albanian Football Federation (FSHF), as evidenced by the team\'s registration at the Balkan Cup tournament during 1929-1931. Despite facing pressure from other teams due to competition, football activities started first and were robust enough for duels. The Albanian National Team was founded on June 6, 1930, but had to wait until 1946 to play its first international match against Yugoslavia, which they won. Albania joined FIFA in 1932 during the 12th-16th June convention and was one of the founding members of UEFA

In [ ]:
# np.save('disambiguated_reference_list_hotpot_100.npy', s.disambiguated_reference_list)    # .npy extension is added if not given
# disabiguated_reference_liit = np.load('disambiguated_reference_list.npy')

In [ ]:
# np.save('embedded_reference_list_hotpot_100.npy', s.embedded_reference)    # .npy extension is added if not given
# embedded_reference = np.load('embedded_reference.npy')

In [23]:
s_demo.disambiguated_reference_list

['radio city',
 'india',
 'it',
 'abraham thomas',
 'albania',
 'fshf team',
 'the teams',
 'nechosmith',
 'indie',
 'chino , california',
 'siblings',
 'noah',
 'graham sierota',
 'jamie',
 'echosmith',
 'warner bros records',
 'the riaa',
 'aria',
 'the song',
 "nwomen ' s colleges",
 'the southern united states',
 'women',
 "girls ' seminaries",
 'academies',
 'salem college institution south',
 'arthur county courthouse',
 'jail',
 'court house',
 'united states s magazine',
 'philadelphia',
 't s . arthur',
 'edgar a . poe',
 'sarah josepha hale',
 "godey ' s lady ' s book",
 'ukrainian',
 'hk kremenchukt',
 'bauer']

In [26]:
from py2neo import Graph, Node, Relationship


# Connect to the Neo4j database
graph = Graph("neo4j://localhost:7687", auth=("neo4j", "password"))
graph.delete_all()

In [27]:
from py2neo import Graph, Node, Relationship


# Connect to the Neo4j database
graph = Graph("neo4j://localhost:7687", auth=("neo4j", "password"))
graph.delete_all()

triplets = kg_demo.triplets

# Create the nodes and relationships
for t in triplets:
    # Create the subject node
    subject, predicate, obj = t[0],t[1],t[2]
    context = None
    if len(t) == 4:
        context = t[3]
    
    try:
        n = graph.run(f"MATCH (n:`{subject}`) RETURN n;").data()
    except Exception as e:
        print(f"Problem with subject {subject}")
        continue

    if n:
        subject_node = n[0]["n"]
    else:
        subject_node = Node(subject, id=subject)
        graph.create(subject_node)

    if predicate == "filepath":
        # obj = re.sub("NYT Files", "'NYT Files'",obj)
        obj = "```"+obj+"```"
        context = retrieve_text_from_filepath(obj[3:-3])
        n = graph.run(f"MATCH (n:{obj}) RETURN n;").data()
        if n:
            # print(n[0]["n"])
            object_node = n[0]["n"]
        else:
            # obj = re.sub("'NYT Files'", "NYT Files",obj)[1:-1]
            obj = obj[2:-2]
            object_node = Node(obj, id=obj)
            graph.create(object_node)
            
    else:
        try:
            n = graph.run(f"MATCH (n:`{obj}`) RETURN n;").data() #I had the assumption that subjects are being linked to wrong entities. Seems that is not the case as it exactly matches object if it exists and not the most similar object
        except Exception as e:
            print(f"Problem with object - {obj}")
            continue
            
        if n:
            object_node = n[0]["n"]
        else:
            object_node = Node(obj, id=obj, text=context)
            graph.create(object_node)

    relationship = Relationship(subject_node, predicate, object_node, text=context)
    graph.merge(relationship)

print("Triplets added to the Neo4j database.")

Triplets added to the Neo4j database.


In [28]:
s = SemanticSimilarity()
s.disambiguated_reference_list

[]

In [29]:
np.save('disambiguated_reference_list_hotpot_4.npy', s_demo.disambiguated_reference_list)    # .npy extension is added if not given
np.save('embedded_reference_kg_demo_hotpot_4.npy', s_demo.embedded_reference)    # .npy extension is added if not given


In [30]:
kg_demo.triplets

[('India', 'has', 'first private FM radio station'),
 ('Radio City', 'is', "India 's FM radio station"),
 ('Radio City', 'was', 'started'),
 ('Radio City', 'is', "India 's first FM radio station"),
 ('Radio City', 'was started on', '3 July 2001'),
 ('Radio City', 'is', "India 's first private FM radio station"),
 ('Radio City', 'is', "India 's private FM radio station"),
 ('2003', 'since 91.1 is', 'Lucknow'),
 ('2003', 'since 91.1 is', 'Bengaluru'),
 ('It', 'broadcasts on', 'Bengaluru'),
 ('It', 'plays', 'Hindi'),
 ('It', 'was', 'launched'),
 ('It', 'was', 'launched in Chennai on 7 July 2006'),
 ('It', 'was launched in', 'Visakhapatnam October 2007'),
 ('It', 'was', 'launched in Hyderabad in March 2006 on 7 July 2006'),
 ('It', 'was', 'launched on 7 July 2006 in Visakhapatnam October 2007'),
 ('It',
  'was',
  'launched in March 2006 in Chennai on 7 July 2006 in Visakhapatnam October 2007'),
 ('It',
  'was',
  'launched in Hyderabad in March 2006 in Chennai in Visakhapatnam October 200

In [31]:
from collections import defaultdict
visited_subjects = defaultdict(list)
for t in kg_demo.triplets:
    s,r,o = t
    if r != "category":
        visited_subjects[s].append(t)

visited_subjects




defaultdict(list,
            {'India': [('India', 'has', 'first private FM radio station')],
             'Radio City': [('Radio City', 'is', "India 's FM radio station"),
              ('Radio City', 'was', 'started'),
              ('Radio City', 'is', "India 's first FM radio station"),
              ('Radio City', 'was started on', '3 July 2001'),
              ('Radio City', 'is', "India 's first private FM radio station"),
              ('Radio City', 'is', "India 's private FM radio station"),
              ('Radio City',
               'forayed with',
               'launch of music PlanetRadiocity.com'),
              ('Radio City', 'recently forayed with', 'launch'),
              ('Radio City',
               'recently forayed with',
               'launch of music PlanetRadiocity.com'),
              ('Radio City',
               'forayed with',
               'launch of music portal PlanetRadiocity.com'),
              ('Radio City', 'recently forayed into', 'New Media'),

In [32]:
s_tr = SemanticSimilarity()
spt = []
for v in kg_demo.triplets:
    spt.append(" ".join(v))
s_tr.generate_reference(spt)

IndentationError: expected an indented block after 'if' statement on line 4 (595322544.py, line 6)

In [35]:
visited = []
s_tr = SemanticSimilarity()

for t in kg_demo.triplets:
    if t[0] in visited:
        continue
    visited.append(t[0])
    triplets_v = visited_subjects[t[0]]
    spliced_triplets = []
    for v in triplets_v:
        spliced_triplets.append(" ".join(v))

    s_tr.generate_reference(spliced_triplets)
    if len(triplets_v) > 1:
        
        print("here is the disambiguated list")
        print(s_tr.disambiguated_reference_list)
    

    

here is the disambiguated list
['India has first private FM radio station', "Radio City is India 's FM radio station"]
here is the disambiguated list
['India has first private FM radio station', '2003 since 91.1 is Lucknow']
here is the disambiguated list
['India has first private FM radio station', '2003 since 91.1 is Lucknow', 'It plays Hindi', 'It was launched', 'It was launched in Chennai on 7 July 2006']
here is the disambiguated list
['India has first private FM radio station', '2003 since 91.1 is Lucknow', 'It plays Hindi', 'It was launched', 'It was launched in Chennai on 7 July 2006', 'Radio station currently plays mix']
here is the disambiguated list
['India has first private FM radio station', '2003 since 91.1 is Lucknow', 'It plays Hindi', 'It was launched', 'It was launched in Chennai on 7 July 2006', 'Radio station currently plays mix', 'Abraham Thomas is CEO of company']
here is the disambiguated list
['India has first private FM radio station', '2003 since 91.1 is Luckn

In [ ]:
x = np.array([[1,3],[2,4]])
x.shape[0]

In [38]:
kg_demo.triplets

[('India', 'has', 'first private FM radio station'),
 ('Radio City', 'is', "India 's FM radio station"),
 ('Radio City', 'was', 'started'),
 ('Radio City', 'is', "India 's first FM radio station"),
 ('Radio City', 'was started on', '3 July 2001'),
 ('Radio City', 'is', "India 's first private FM radio station"),
 ('Radio City', 'is', "India 's private FM radio station"),
 ('2003', 'since 91.1 is', 'Lucknow'),
 ('2003', 'since 91.1 is', 'Bengaluru'),
 ('It', 'broadcasts on', 'Bengaluru'),
 ('It', 'plays', 'Hindi'),
 ('It', 'was', 'launched'),
 ('It', 'was', 'launched in Chennai on 7 July 2006'),
 ('It', 'was launched in', 'Visakhapatnam October 2007'),
 ('It', 'was', 'launched in Hyderabad in March 2006 on 7 July 2006'),
 ('It', 'was', 'launched on 7 July 2006 in Visakhapatnam October 2007'),
 ('It',
  'was',
  'launched in March 2006 in Chennai on 7 July 2006 in Visakhapatnam October 2007'),
 ('It',
  'was',
  'launched in Hyderabad in March 2006 in Chennai in Visakhapatnam October 200

In [39]:
triples = []
for k in kg_demo.triplets:
    t = list(k)
    if len(t) == 3:
        t = t + [""]
    if len(t) > 4:
        continue
    triples.append(t)



In [40]:
triples

[['India', 'has', 'first private FM radio station', ''],
 ['Radio City', 'is', "India 's FM radio station", ''],
 ['Radio City', 'was', 'started', ''],
 ['Radio City', 'is', "India 's first FM radio station", ''],
 ['Radio City', 'was started on', '3 July 2001', ''],
 ['Radio City', 'is', "India 's first private FM radio station", ''],
 ['Radio City', 'is', "India 's private FM radio station", ''],
 ['2003', 'since 91.1 is', 'Lucknow', ''],
 ['2003', 'since 91.1 is', 'Bengaluru', ''],
 ['It', 'broadcasts on', 'Bengaluru', ''],
 ['It', 'plays', 'Hindi', ''],
 ['It', 'was', 'launched', ''],
 ['It', 'was', 'launched in Chennai on 7 July 2006', ''],
 ['It', 'was launched in', 'Visakhapatnam October 2007', ''],
 ['It', 'was', 'launched in Hyderabad in March 2006 on 7 July 2006', ''],
 ['It', 'was', 'launched on 7 July 2006 in Visakhapatnam October 2007', ''],
 ['It',
  'was',
  'launched in March 2006 in Chennai on 7 July 2006 in Visakhapatnam October 2007',
  ''],
 ['It',
  'was',
  'launc

In [41]:
np.save('hotpot_4_triplets.npy', np.array(triples))